In [1]:
import requests
import base64
import sys

In [2]:
CLIENT_ID = '9cbb2e201276412ea1a06d2e23ef5c65'
CLIENT_SECRET = '12c52f0591de4da4b18d66119e24c36c'

In [3]:
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

In [4]:
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}',
    'Content-Type': 'application/x-www-form-urlencoded'
}
data = {
    'grant_type': 'client_credentials'
}
try:
    response = requests.post(token_url, data=data, headers=headers)
    if response.status_code == 200:
        access_token = response.json()['access_token']
        print("Access token obtained successfully:", access_token)
    else:
        print(f"Error obtaining access token: {response.status_code} - {response.reason}")
        print("Response:", response.text)
        sys.exit(1)
except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")
    sys.exit(1)

Access token obtained successfully: BQAZTO3tcbXltum20WBMK2wD3KYSXv2gZ6niJ9MIgk-S8aciWGfpawz7ZRnJgoQ9jQzZ9aT0uuyeZcX5KTgvEwH8KyrQ--f-DVIwGqvKhSE9j2nHC8Y


In [5]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

In [6]:
def get_trending_playlist_data(playlist_id, access_token):
    sp = spotipy.Spotify(auth=access_token)
    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))')
    music_data = []
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']
        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None
        try:
            album_info = sp.album(album_id) if album_id != 'Not available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None
        try:
            track_info = sp.track(track_id) if track_id != 'Not available' else None
            popularity = track_info['popularity'] if track_info else None
        except:
            popularity = None
        track_data = {
            'Track Name': track_name,
            'Artists': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Release Date': release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit': track_info.get('explicit', None),
            'External URLs': track_info.get('external_urls', {}).get('spotify', None),
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,
        }
        music_data.append(track_data)
    df = pd.DataFrame(music_data)
    return df

In [7]:
playlist_id = '6M14oz8xWgu8XzDv5OQB5Y'
music_df = get_trending_playlist_data(playlist_id, access_token)
print(music_df)

            Track Name                                       Artists  \
0              Perfect                                    Ed Sheeran   
1         Shape of You                                    Ed Sheeran   
2    Thinking out Loud                                    Ed Sheeran   
3           Photograph                                    Ed Sheeran   
4   Castle on the Hill                                    Ed Sheeran   
..                 ...                                           ...   
95  Kissing Undercover                                      Ladybyrd   
96             Freedom  BEX, Rebecca Houlihan, Captain's Brotherhood   
97             Tell Me                                    Kasey Jane   
98            Run Away                              Xi Aquarii, K1NA   
99           Safe 2024                            Paul in the Short5   

            Album Name                Album ID                Track ID  \
0           ÷ (Deluxe)  3T4tUhGYeRNVUGevb0wThu  0tgVpDi06FyKp

In [8]:
print(music_df.isnull().sum())

Track Name          0
Artists             0
Album Name          0
Album ID            0
Track ID            0
Popularity          0
Release Date        0
Duration (ms)       0
Explicit            0
External URLs       0
Danceability        0
Energy              0
Key                 0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
dtype: int64


In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

data = music_df

In [10]:
def calculate_weighted_popularity(release_date):
    release_date = datetime.strptime(release_date, '%Y-%m-%d')
    time_span = datetime.now() - release_date
    weight = 1 / (time_span.days + 1)
    return weight

In [11]:
scaler = MinMaxScaler()
music_features = music_df[['Danceability', 'Energy', 'Key', 
                           'Loudness', 'Mode', 'Speechiness', 'Acousticness',
                           'Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values
music_features_scaled = scaler.fit_transform(music_features)

In [12]:
def content_based_recommendations(input_song_name, num_recommendations=5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return
    input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]
    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)
    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]
    content_based_recommendations = music_df.iloc[similar_song_indices][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]
    return content_based_recommendations

In [13]:
import pandas as pd
def hybrid_recommendations(input_song_name, num_recommendations=5, alpha=0.5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return
    content_based_rec = content_based_recommendations(input_song_name, num_recommendations)
    popularity_score = music_df.loc[music_df['Track Name'] == input_song_name, 'Popularity'].values[0]
    weighted_popularity_score = popularity_score * calculate_weighted_popularity(
        music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0])
    new_entry = pd.DataFrame({
        'Track Name': [input_song_name],
        'Artists': [music_df.loc[music_df['Track Name'] == input_song_name, 'Artists'].values[0]],
        'Album Name': [music_df.loc[music_df['Track Name'] == input_song_name, 'Album Name'].values[0]],
        'Release Date': [music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0]],
        'Popularity': [weighted_popularity_score]
    })
    hybrid_recommendations = pd.concat([content_based_rec, new_entry], ignore_index=True)
    hybrid_recommendations = hybrid_recommendations.sort_values(by='Popularity', ascending=False)
    hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['Track Name'] != input_song_name]
    return hybrid_recommendations

In [14]:
input_song_name = "Perfect"
recommendations = hybrid_recommendations(input_song_name, num_recommendations=5)
print(f"Hybrid recommended songs for '{input_song_name}':")
print(recommendations)

Hybrid recommended songs for 'Perfect':
                                       Track Name                     Artists  \
1  Perfect Symphony (Ed Sheeran & Andrea Bocelli)  Ed Sheeran, Andrea Bocelli   
0                      How Would You Feel (Paean)                  Ed Sheeran   
3               Put It All on Me (feat. Ella Mai)        Ed Sheeran, Ella Mai   
2                                           Tides                  Ed Sheeran   
4                                      Afire Love                  Ed Sheeran   

                                       Album Name Release Date  Popularity  
1  Perfect Symphony (Ed Sheeran & Andrea Bocelli)   2017-12-15        65.0  
0                                      ÷ (Deluxe)   2017-03-03        63.0  
3                     No.6 Collaborations Project   2019-07-12        59.0  
2                                               =   2021-10-25        57.0  
4                              x (Deluxe Edition)   2014-06-21        55.0  
